In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import make_pipeline 

In [7]:
df = pd.read_csv('data_train.csv')
df.head()

,Input_current,Output_voltage,Input_voltage,Duty_Cycle,Power,Irradiance,Temperature
0,6.818701,58.373037,29.782506,0.485,212.117783,1000,25
1,7.313437,53.815293,23.990136,0.557,180.751131,1000,60
2,6.056522,49.442379,24.298210,0.505,152.417456,836,60
3,5.735578,50.894729,26.911060,0.475,161.503052,836,48
4,5.376322,48.436413,26.198545,0.457,146.294526,755,48


Separamos as 4 entradas: Irradiance, Temperature, Tensão de Entrada e Corrente de entrada
Separamos a saida: DutyCycle

In [8]:
df_x = df.loc[:,['Irradiance','Temperature','Input_current','Input_voltage']] #Seleciona por nome
df_y = df.loc[:,['Duty_Cycle']] #Seleciona por nome
print(df_x[:5])
print(df_y[:5])
print(df_x.shape)

   Irradiance  Temperature  Input_current  Input_voltage
0        1000           25       6.818701      29.782506
1        1000           60       7.313437      23.990136
2         836           60       6.056522      24.298210
3         836           48       5.735578      26.911060
4         755           48       5.376322      26.198545
   Duty_Cycle
0       0.485
1       0.557
2       0.505
3       0.475
4       0.457
(2093, 4)


In [9]:
x=np.asarray(df_x)
y=np.asarray(df_y)
print(x.shape,y.shape)

(2093, 4) (2093, 1)


Após a obtenção dos dados do arquivo, é feito a separação do conjunto de dados entre treinamento e teste.

In [10]:
X_train, X_test, y_train, y_test = train_test_split(x, y,test_size=0.5,random_state=1)

Em seguida, cria-se um modelo para a normalização dos dados para que se obtenha os dados com media zero e variancia unitaria. Assim  são normalizados os dados de entrada de treinamento e de teste.

In [11]:
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [12]:
# Now apply the transformations to the data:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
print(y_train.shape)

(1046, 1)


Em seguida são levantados os parametros desejados da rede neural e feito o seu treinamento

In [19]:
modelo = MLPRegressor(activation='tanh', solver='lbfgs', hidden_layer_sizes=(4,4))
modelo.fit(X_train,y_train)
print(modelo.score(X_test, y_test))

print(modelo.n_layers_)
print(modelo.coefs_)

0.9971909278630618
4
[array([[-0.65222734, -0.27083855,  0.14035017,  0.33373584],
       [ 0.16450235,  0.07311607,  0.21305128,  0.44776445],
       [ 0.32144287,  0.587703  , -0.16228334,  0.8463759 ],
       [ 0.34007996,  0.0284399 ,  0.43214021,  0.04023597]]), array([[-0.50123231, -0.06025406, -0.48308484,  0.4188902 ],
       [ 0.69181075, -0.61042139,  0.39633781, -0.52256896],
       [ 0.46400971,  0.75235055,  0.40672267,  0.06301714],
       [ 0.19923661,  0.27081752, -0.08771699, -0.12767324]]), array([[ 0.32334741],
       [-0.4869193 ],
       [ 0.57908447],
       [-0.18620574]])]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:1306: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Geração de arquivos .csv com os valores de pesos e bias do modelo criado para import no matlab

In [27]:
coef_0=np.asarray(modelo.coefs_[0])
coef_1=np.asarray(modelo.coefs_[1])
coef_2=np.asarray(modelo.coefs_[2])
coefs=np.concatenate((coef_0,coef_1),axis=1)
coefs=np.concatenate((coefs,coef_2),axis=1)
np.savetxt("pesos.csv", coefs, delimiter=",")

bias_0=np.asarray(modelo.intercepts_[0])
bias_1=np.asarray(modelo.intercepts_[1])
bias_2=np.asarray(modelo.intercepts_[2])
bias=np.concatenate((bias_0,bias_1))
bias=np.concatenate((bias,bias_2))
np.savetxt("bias.csv", bias, delimiter=",")
print(modelo.intercepts_)

[array([-0.11386737,  1.15618983, -0.6126903 ,  0.74691495]), array([ 0.64267996, -0.40288471,  0.59993693, -0.79807246]), array([-0.74480622])]


In [28]:
y_hat = mlp.predict(X_test)
y_hat[0]

0.3700510594573566

In [422]:
y_test[0]

array([0.375])

In [423]:
np.mean((y_hat-y_test)**2)

0.02111335787845592

In [424]:
valid = pd.read_csv('data.csv')
valid.head()

,Input_current,Output_voltage,Input_voltage,Duty_Cycle,Power,Irradiance,Temperature
0,6.818671,58.373244,29.782580,0.485,212.118655,1000.0,25
1,7.206004,57.995148,28.079099,0.513,210.170608,1000.0,27
2,7.651242,59.317953,27.550531,0.537,220.197268,1048.0,27
3,7.030870,60.620215,31.017551,0.483,228.780529,1048.0,17
4,4.054196,45.744741,30.788208,0.325,130.447698,588.0,17


In [425]:
valid_x = valid.loc[:,['Irradiance','Temperature','Input_current','Input_voltage']] #Seleciona por nome
valid_y = valid.loc[:,['Duty_Cycle']] #Seleciona por nome
print(valid_x[:5])
print(valid_y[:-1])
print(valid_x.shape)

   Irradiance  Temperature  Input_current  Input_voltage
0      1000.0           25       6.818671      29.782580
1      1000.0           27       7.206004      28.079099
2      1048.0           27       7.651242      27.550531
3      1048.0           17       7.030870      31.017551
4       588.0           17       4.054196      30.788208
      Duty_Cycle
0          0.485
1          0.513
2          0.537
3          0.483
4          0.325
5          0.361
6          0.529
7          0.501
8          0.117
9          0.179
10         0.417
11         0.403
12         0.503
13         0.465
14         0.507
15         0.431
16         0.385
17         0.377
18         0.411
19         0.487
20         0.505
21         0.475
22         0.487
23         0.517
24         0.389
25         0.315
26         0.419
27         0.437
28         0.517
29         0.469
...          ...
1969       0.000
1970       0.000
1971       0.000
1972       0.000
1973       0.000
1974       0.000
1975       0

In [426]:
x_val=np.asarray(valid_x)
y_val=np.asarray(valid_y)
print(x_val.shape,y_val.shape)

(2000, 4) (2000, 1)


In [427]:
X_val = scaler.transform(x_val)
mlp.score(X_val,y_val)

-2.2756861389438323

In [428]:
Y_val=mlp.predict(X_val)
print(Y_val)

[0.487382   0.51760236 0.54462681 ... 0.54519682 0.54519682 0.54519682]


In [429]:
y_val


array([[0.485],
       [0.513],
       [0.537],
       ...,
       [0.   ],
       [0.   ],
       [0.   ]])

In [430]:
np.mean((Y_val-y_val)**2)

0.1315794442522724

In [431]:
print('Accuracy testing : {:.3f}'.format(mlp.score(X_val, y_val)))

Accuracy testing : -2.276
